In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from datetime import timezone
from dateutil import parser

# Show all columns when printing a pandas DataFrame
pd.options.display.max_columns = None

In [ ]:
# Craetes a list of the fileds with a file as input
def read_fields(filename):
    fields = []
    fd = open(filename)
    lines = fd.readlines()
    for line in lines:
        fields.append("data."+line[:-1])
    fd.close()
    return fields

def remove_dot_data(fields):
    fields_no_data=[]
    for field in fields:
        field_no_data = field.replace("data.","")
        fields_no_data.append(field_no_data)
    return fields_no_data

def read_token(filename):
    fd = open(filename)
    line = fd.readline()
    # Remove the 'newline' character if found
    if line[len(line)-1] == '\n':
        line = line[:-1]
    fd.close()
    return line

In [ ]:
# Where to get the authorization token
file_token="./token"

# Where to get the list of fields to retrieve
file_fields="./fields"

# Read the authorization token
token = read_token(file_token)
headers = {'Authorization': 'Bearer '+token,}

# Set the start(min) and end(max) dates to perform the query onto
min_date_human_readable = "Jun 6 10:00:00 UTC 2019"
max_date_human_readable = "Jun 6 10:59:00 UTC 2019"

# Get timestamps in miliseconds
min_date = str(int(parser.parse(min_date_human_readable).timestamp() * 1000))
max_date = str(int(parser.parse(max_date_human_readable).timestamp() * 1000))

# The query on lucene syntaxis
query =               "data.Type:analysis"
query = query + " AND  data.Status:Completed"
query = query + " AND  data.JobUniverse:5"

# Read the list of fields to retrieve from a file
fields = read_fields(file_fields)
fields_no_data = remove_dot_data(fields)

# Number of records to retreive
num_records = 100

In [ ]:
print("Query:")
print(query)
print("Fields no data:")
print(fields_no_data)

data = {
        "size":num_records,
        "query":{
            "bool":{
                "filter":[
                    {"range":{"data.RecordTime":{"gte":min_date,"lte":max_date,"format":"epoch_millis"}}},
                    {"query_string":{
                        "analyze_wildcard":"true",
                        "query":query
                        }
                    }
                ]
            }
        },
        "_source":fields
}

data_string = json.dumps(data)

In [ ]:
# Send the query (It takes few seconds)
response = requests.get('https://monit-grafana.cern.ch/api/datasources/proxy/9014/_search', headers=headers, data=data_string)


In [ ]:
# Get the data from the response
d = response.json()
# Create a pandas DataFrame with the data retreived
clean_records=[]
for record in d['hits']['hits']:
    clean_record= record['_source']['data']
    clean_records.append(clean_record)
df = pd.DataFrame(clean_records) 

In [ ]:
# Print the DataFrame
df